In [1]:
import re
import httpx
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from groq import Groq
client = Groq(api_key=os.getenv("GROQ_API_KEY"))
class GroqLLM:
    def __init__(self, client, model):
        self.client = client
        self.model = model

    def __call__(self, query):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": query}]
        )
        return response.choices[0].message.content

# Initialize like this
llm = GroqLLM(client, "llama-3.3-70b-versatile")

In [3]:
print(llm("Hello World"))

Hello. Is there something I can help you with or would you like to chat?


In [20]:
class Agent:
    def __init__(self,system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role":"system","content":system})
    
    def __call__(self, message):
        self.messages.append({"role":"user","content":message})
        result = self.execute()
        self.messages.append({"role":"assistant","content":result})
        return result
    
    def execute(self):
        client = Groq(api_key=os.getenv("GROQ_API_KEY"))
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=self.messages
        )
        return response.choices[0].message.content

In [21]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [22]:
abot = Agent(prompt)
abot

In [23]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [24]:
response = abot("How much does a toy poodle weigh?")

In [25]:
print(response)

Thought: I should look up the weight of a toy poodle using the average_dog_weight action to find the average weight of this breed.

Action: average_dog_weight: Toy Poodle
PAUSE


In [30]:
answer = average_dog_weight("Toy Poodle")
response_prompt = "Observation : {}".format(answer)

In [31]:
print(abot(response_prompt))

Thought: The observation has now provided the average weight of a Toy Poodle, which is 7 lbs. This information directly answers the question about the weight of a Toy Poodle.

Answer: A Toy Poodle weighs 7 lbs.


In [32]:
question = """I have two dogs, a border coolie and a scottish terrier. What is their combined weight"""
response = abot(question)
print(response)

Thought: To find the combined weight of the two dogs, I need to know the average weight of a Border Collie and a Scottish Terrier. I can use the average_dog_weight action to find these weights.

Action: average_dog_weight: Border Collie
PAUSE


In [34]:
weight = average_dog_weight("Border Collie")
prompt = "Observation : {}".format(weight)
print(abot(prompt))

Thought: Now that I have the average weight of a Border Collie, I need to find the average weight of a Scottish Terrier to calculate their combined weight.

Action: average_dog_weight: Scottish Terrier
PAUSE


In [35]:
weight2 = average_dog_weight("Scottish Terrier")
prompt = "Observation : {}".format(weight2)
print(abot(prompt))

Thought: With the average weights of both the Border Collie (37 lbs) and the Scottish Terrier (20 lbs), I can now calculate their combined weight.

Action: calculate: 37.0 + 20.0
PAUSE


In [36]:
next_prompt = "Observation : {}".format(eval("37+20"))
print(abot(next_prompt))

Thought: The calculation has returned the combined weight of the two dogs, which is 57 lbs.

Answer: The combined weight of the Border Collie and the Scottish Terrier is 57 lbs.
